# 6.4 多输入多输出通道
- **目录**
  - 6.4.1 多输入通道
  - 6.4.2 多输出通道
  - 6.4.3 $1\times 1$ 卷积层

虽然我们在6.1.4.1节中描述了构成每个图像的多个通道和多层卷积层。例如彩色图像具有标准的RGB通道来代表红、绿和蓝。
但是到目前为止，我们仅展示了单个输入和单个输出通道的简化例子。
这使得我们可以将输入、卷积核和输出看作二维张量。

当我们添加通道时，我们的输入和隐藏的表示都变成了三维张量。例如，每个RGB输入图像具有$3\times h\times w$的形状。我们将这个大小为$3$的轴称为**通道（channel）维度**。在本节中，我们将更深入地研究具有多输入和多输出通道的卷积核。

- **要点：**
  - **多通道图像**：例如，标准的彩色图像有RGB三个颜色通道，表示红色、绿色和蓝色。
  - **三维张量**：添加通道后，图像表示从二维张量（宽度和高度）变为三维张量（通道、宽度和高度）。例如，RGB图像具有$3 \times h \times w$的形状。
  - **通道维度**：三维张量中大小为$3$的轴代表不同的颜色通道，称为通道维度。
  - **多通道卷积**：在具有多个输入和输出通道的情况下，卷积操作需考虑输入中每个通道的卷积核，以及所有通道的输出结果。

## 6.4.1 多输入通道

- 当输入包含多个通道时，需要构造一个与输入数据具有**相同输入通道数**的卷积核，以便与输入数据进行**互相关运算**。
- 假设输入的通道数为$c_i$，那么卷积核的输入通道数也需要为$c_i$。
- 如果卷积核的窗口形状是$k_h\times k_w$，那么当$c_i=1$时，就将卷积核看作形状为$k_h\times k_w$的二维张量。

- 当$c_i>1$时，卷积核的每个输入通道将包含形状为$k_h\times k_w$的张量，将这些张量$c_i$连结在一起可以得到形状为$c_i\times k_h\times k_w$的卷积核。
- 由于输入和卷积核都有$c_i$个通道，可以对每个通道输入的二维张量和卷积核的二维张量进行互相关运算，再对通道求和（将$c_i$的结果相加）得到二维张量。这是多通道输入和多输入通道卷积核之间进行二维互相关运算的结果。

- 图6.4.1中演示了两个输入通道的二维互相关运算的示例。
  - 阴影部分是第一个输出元素以及用于计算这个输出的输入和核张量元素：$(1\times1+2\times2+4\times3+5\times4)+(0\times0+1\times1+3\times2+4\times3)=56$。

<center>
    <img src="../img/conv-multi-in.svg" alt="两个输入通道的互相关计算">
</center>
<center>
    图6.4.1 两个输入通道的互相关计算
</center>


- 简而言之，就是对每个通道执行互相关操作，然后将结果相加。

In [10]:
import torch
from d2l import torch as d2l

In [11]:
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    '''
    多通道X张量输入与多个卷积核对应进行卷积计算，
    输出的张量是对每个对应张量与卷积核进行卷积计算之后的张量按元素求和之后得到一个张量（矩阵）。
    sum函数是对多个形状相同的张量按元素求和，另外，d2l.reduce_sum和numpy.sum以及标准库的sum函数，
    表现形式存在差别。
    '''
        
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

- 可以构造与 图6.4.1中的值相对应的输入张量`X`和核张量`K`，以验证互相关运算的输出。


In [12]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

- 使用Pytorch的Conv2d。

In [13]:
'''
使用torch的Conv2d。
卷积核使用均匀分布自动初始化。
'''
conv2d=torch.nn.Conv2d(2,1,(2,2),bias=True)
output=conv2d(X)
'''
卷积计算的输出，以及卷积核（权重）和偏置。
后两个参数是需要训练学习的对象。
'''
output,conv2d.weight,conv2d.bias

(tensor([[[-2.2531, -3.3282],
          [-5.4785, -6.5536]]], grad_fn=<SqueezeBackward1>),
 Parameter containing:
 tensor([[[[-0.3238, -0.2282],
           [-0.0639, -0.3068]],
 
          [[ 0.1176, -0.0368],
           [-0.2024, -0.0308]]]], requires_grad=True),
 Parameter containing:
 tensor([0.3135], requires_grad=True))

In [14]:
# weight的个维度分别表示输出通道、输入通道、高度和宽度
conv2d.weight.shape

torch.Size([1, 2, 2, 2])

- conv2d.weight的形状torch.Size([1, 2, 2, 2])详细说明：
  - 1 个卷积核（因为输出通道数是 1）。
  - 每个卷积核有 2 个输入通道（因为输入通道数是 2）。
  - 每个卷积核的大小是 2x2。

## 6.4.2 多输出通道

- 正如6.1.4.1节中所讨论的，每一层有多个输出通道是至关重要的。
- 在最流行的神经网络架构中，随着神经网络层数的加深，常会增加输出通道的维数，通过**减少空间分辨率**以获得更大的通道深度。
  - 更丰富的表征能力：增加输出通道数意味着每层都可以学习更多的特征。这使得神经网络能够捕捉到输入数据中更复杂、更抽象的特征，从而提高模型的表征能力。
    - 低级特征：在网络的初始层，学习到的特征通常与输入数据的基本可视属性有关。例如，在图像分类任务中，这些特征可能包括边缘、角点、颜色变化等。这些特征有助于捕捉图像中的简单结构。
    
    -  中级特征：在网络的中间层，学习到的特征开始变得更加抽象。这些特征通常涉及到局部区域的形状、纹理、特定模式等。例如，在人脸识别任务中，这些特征可能包括眼睛、鼻子、嘴巴等局部结构。
    
    - 高级特征：在网络的深层，学习到的特征更加复杂且**语义丰富**。这些特征能够捕捉到全局信息、对象之间的关系以及更高层次的概念。例如，在场景分类任务中，这些特征可能包括不同物体之间的相对位置、场景中的主要元素等。
  
  - 减少计算成本：通过减少空间分辨率，我们可以降低每一层特征图的尺寸。这将减少后续层所需的计算量，从而提高整个网络的计算效率。
  
  - 控制模型复杂度：随着网络层数的加深，模型的参数数量可能会显著增加。通过降低空间分辨率，我们可以有效地减少参数数量，从而控制模型复杂度，降低过拟合的风险。
  
- 直观地说，可以**将每个通道看作是对不同特征的响应**。

- 现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器。

- 用$c_i$和$c_o$分别表示输入和输出通道的数目，并让$k_h$和$k_w$为卷积核的高度和宽度。
- 为了获得多个通道输出，可为每个输出通道创建一个形状为$c_i\times k_h\times k_w$的卷积核张量，因而卷积核的形状是$c_o\times c_i\times k_h\times k_w$。
- 在互相关运算中，每个输出通道先获取所有输入通道，再以对应该输出通道的卷积核计算出结果。

- **要点：**
  - **多输出通道的重要性**：在神经网络设计中，随着层次深入，增加输出通道的数量是常见的做法，这有助于逐步减小空间分辨率，同时增加通道深度。
  - **表征能力的增强**：通过增加输出通道数量，神经网络能够学习到更多、更复杂、更抽象的特征，从而增强模型的表征能力。
    - **低级特征**：网络的初始层关注于捕捉简单的结构和可视属性，例如边缘、角点、颜色变化等。
    - **中级特征**：网络的中间层学习到更加抽象的局部形状、纹理和特定模式等特征。
    - **高级特征**：网络的深层捕捉到语义丰富的全局信息、对象间关系和高层次概念。
  - **降低计算成本**：通过减小空间分辨率，降低每层特征图的尺寸，从而减少计算量，提高网络的计算效率。
  - **控制模型复杂度**：适当降低空间分辨率有助于减少网络参数数量，控制模型复杂度，降低过拟合风险。
  - **通道间的交互与协作**：每个输出通道可以视为对不同特征的响应器。然而，这些通道并非独立学习，而是相互协作，经过优化以共同提取和表征输入数据的特征。
  - **卷积核形状**：为获取多输出通道，每个输出通道都对应一个$c_i\times k_h\times k_w$形状的卷积核张量，整体卷积核形状为$c_o\times c_i\times k_h\times k_w$。在进行互相关运算时，每个输出通道将综合所有输入通道信息，通过对应的卷积核计算得到结果。

- 下例实现一个**计算多个通道的输出的互相关函数**。

In [15]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起，和前面不一样，此处用的叠加而不是求和
    
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

通过将核张量`K`与`K+1`（`K`中每个元素加$1$）和`K+2`连接起来，构造了一个具有$3$个输出通道的卷积核。


In [17]:
## 三个通道，每个通道有2个卷积核
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [18]:
K

tensor([[[[0., 1.],
          [2., 3.]],

         [[1., 2.],
          [3., 4.]]],


        [[[1., 2.],
          [3., 4.]],

         [[2., 3.],
          [4., 5.]]],


        [[[2., 3.],
          [4., 5.]],

         [[3., 4.],
          [5., 6.]]]])

- 下面对输入张量`X`与卷积核张量`K`执行互相关运算。现在的输出包含$3$个通道，第一个通道的结果与先前输入张量`X`和多输入单输出通道的结果一致。


In [19]:
'''
3个卷积通道按照每个通道调用corr2d_multi_in函数，
再通过corr2d_multi_in调用d2l.corr2d,
最后将3个通道的张量按第1维（0轴）进行堆叠。
'''
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 6.4.3 $1\times 1$ 卷积层

- $1 \times 1$卷积，即$k_h = k_w = 1$，看起来似乎没有多大意义。

  - 因为使用了最小窗口，$1\times 1$卷积失去了卷积层的特有能力——**在高度和宽度维度上，识别相邻元素间相互作用的能力**。
  
- 其实$1\times 1$卷积的**唯一计算发生在通道**上。

- 图6.4.2展示了使用$1\times 1$卷积核与$3$个输入通道和$2$个输出通道的互相关计算。

- 这里输入和输出具有相同的高度和宽度，输出中的每个元素都是从输入图像中同一位置的元素的线性组合。

- 可以将$1\times 1$卷积层看作是在每个像素位置应用的全连接层，以$c_i$个输入值转换为$c_o$个输出值。

- 同时，$1\times 1$卷积层需要的权重维度为$c_o\times c_i$，再额外加上一个偏置。

<center>
    <img src="../img/conv-1x1.svg" alt="互相关计算使用了具有3个输入通道和2个输出通道的 $1\times 1$ 卷积核。其中，输入和输出具有相同的高度和宽度">
</center>
<center>图6.4.2 互相关计算使用了具有3个输入通道和2个输出通道的卷积核。其中，输入和输出具有相同的高度和宽度。</center>

---------

- **说明：$1\times 1$ 卷积层的功能是否仅与调整输出的通道数有关**？
    - $1\times 1$卷积层的功能并**不仅限于调整输出通道数**。虽然它们确实可以在保持输出高度和宽度不变的情况下调整通道数，但它们还具有以下功能。
    - **非线性**：由于$1\times 1$卷积层中的卷积操作后面通常跟随着激活函数（如 ReLU），因此它们可以引入非线性，使得网络可以更好地学习复杂模式。
    - **参数减少**: $1\times 1$卷积层可以用来减少网络参数数量。当输入通道数很大时，可以使用具有**较少输出通道数**的$1\times 1$卷积层来压缩特征表示，从而降低计算复杂性和内存消耗。
    - **特征交互**：尽管$1\times 1$卷积层没有在空间维度上捕捉相邻元素之间的关系，但它们可以学习不同输入通道之间的特征交互。这有助于网络将来自多个输入通道的信息组合起来，从而构建更丰富的特征表示。
 


-------------

- 下例使用**全连接层实现$1 \times 1$卷积**。
  - 请注意，此处需要对输入和输出的数据形状进行调整。

In [20]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

- 当执行$1\times 1$卷积运算时，上述函数相当于先前实现的互相关函数`corr2d_multi_in_out`。
- 下例使用样本数据进行验证。


In [21]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [22]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
#assert float(torch.abs(Y1 - Y2).sum()) < 1e-6
Y1,Y2, Y1.shape, Y2.shape

(tensor([[[ 5.4713,  2.8535, -8.7930],
          [-3.3389,  2.4393, -3.4665],
          [ 1.9439, -4.2926, -0.2690]],
 
         [[ 0.0439,  0.7676,  0.7553],
          [ 0.2858,  0.3787,  0.3000],
          [ 0.5589, -0.0144, -0.5747]]]),
 tensor([[[ 5.4713,  2.8535, -8.7930],
          [-3.3389,  2.4393, -3.4665],
          [ 1.9439, -4.2926, -0.2690]],
 
         [[ 0.0439,  0.7676,  0.7553],
          [ 0.2858,  0.3787,  0.3000],
          [ 0.5589, -0.0144, -0.5747]]]),
 torch.Size([2, 3, 3]),
 torch.Size([2, 3, 3]))

In [23]:
## 数据形状的变换
X.shape,K.shape

(torch.Size([3, 3, 3]), torch.Size([2, 3, 1, 1]))

----------

- **说明：corr2d_multi_in_out_1x1函数解析**

In [48]:
'''
这种算法的基本思路是：
(1)将输入张量X的形状变成每个通道的矩阵(3,3)都是一行(9,)，3个通道的数据就变成(3,9)。
(2)然后将卷积核K变成(2,3)。
(3)在进行K@X，注意K是左乘，这种算法本质上就是使用2个通道的3个1*1卷积核中的单元素与X的3个通道里每个
   通道的对应元素进行乘法运算，最后的结果是(2,9)形状。
(4)然后将(2,9)变形成(2,3,3)，得到最终结果。
'''
## 按照这个方法进行运算的形状变换
## X的形状：(3,3,3)是均值为0，标准差为1的正态分布随机数
X = torch.normal(0, 1, (3, 3, 3))
#print('X.value:',X)
## 2通道，3个1*1卷积核
K = torch.normal(0, 1, (2, 3, 1, 1))
## c_i,h,w的值等于3
c_i, h, w = X.shape
## c_o=2
c_o = K.shape[0]
## X的形状变成(3,9)
X = X.reshape((c_i, h * w))
## K的形状编程(2,3)
K = K.reshape((c_o, c_i))

## 全连接层中的矩阵乘法，应该是人为设计出来的快捷算法
## 本操作是(2,3)@(3,9)=(2,9)
Y = torch.matmul(K, X)
##再将Y变形为(2,3,3)
Y.reshape((c_o, h, w))

tensor([[[-0.4774,  2.9668,  1.9555],
         [-3.4611, -2.7919,  1.7293],
         [-0.3247, -1.4660, -2.7230]],

        [[-0.6237, -0.4855, -0.3349],
         [-0.7839, -0.7294, -0.5025],
         [ 0.6954, -4.9168, -3.4021]]])

---------

## 小结

* 多输入多输出通道可以用来扩展卷积层的模型。
* 当以每像素为基础应用时，$1\times 1$卷积层相当于全连接层。
* $1\times 1$卷积层通常用于调整网络层的通道数量和控制模型复杂性。

